# Классификация новостей

In [ ]:
%%capture
!wget -O lenta-ru-train.csv https://www.dropbox.com/scl/fi/xv7ycyd56omonay46co7p/lenta-ru-train.csv?rlkey=bfgu5bh5r340rddwla4b1hu4l&dl=0
!wget -O lenta-ru-test.csv https://www.dropbox.com/scl/fi/3qd36eoedlfqmzg9hm446/lenta-ru-test.csv?rlkey=8qvtm2o1xdjpg365pdgys9po8&dl=0

### Загрузка данных

In [ ]:
import re
import pandas as pd

In [ ]:
data_train = pd.read_csv('lenta-ru-train.csv')
data_test = pd.read_csv('lenta-ru-test.csv')

In [ ]:
data_train.head()

title  \
0          В Diablo III появились бои между игроками   
1     Бернанке пригласили старшим советником в PIMCO   
2  Чешские футболисты разгромили боснийскую гости...   
3   Михалков возглавит жюри Пекинского кинофестиваля   
4    Росатом получил контроль над Атомстройэкспортом   

                                                text            topic  \
0  Студия Blizzard добавила в Diablo III сражения...  Наука и техника   
1  Бывший председатель Федеральной резервной сист...        Экономика   
2  Футболисты молодежной сборной Чехии, которые в...            Спорт   
3  Режиссер Никита Михалков возглавит жюри наград...         Культура   
4  Глава Росатома Сергей Кириенко заявил, что гос...        Экономика   

   topic_label                                              clean  
0            2  студия blizzard добавила в diablo iii сражения...  
1            0  бывший председатель федеральной резервной сист...  
2            1  футболисты молодежной сборной чехии которые в ...  
3            3  режиссер никита михалков возглавит жюри наград...  
4            0  глава росатома сергей кириенко заявил что госу...

In [ ]:
# убираем все небуквенные символы
regex = re.compile("[А-Яа-я:=!\)\()A-z\_\%/|]+")

def words_only(text, regex=regex):
    try:
        return " ".join(regex.findall(text.lower()))
    except:
        return ""

In [ ]:
%%time
data_train['clean'] = data_train['text'].apply(words_only)
data_test['clean'] = data_test['text'].apply(words_only)

CPU times: user 4.89 s, sys: 127 ms, total: 5.02 s
Wall time: 5.03 s


### Готовим два текстовых файла:
* файл с тренировочными данными
* файл для оценки решения в kaggle (необходимо подготовить данные в таком виде для модели fasttext)

In [ ]:
X_train = data_train['clean'].values
y_train = data_train['topic_label'].values
with open('train.txt', 'w+') as train_file:
    for i in range(len(X_train)):
        train_file.write('__label__' + str(y_train[i]) + ' '+ X_train[i] + '\n')


X_test = data_test['clean'].values
with open('test.txt', 'w+') as test_file:
    for i in range(len(X_test)):
        test_file.write(X_test[i] + '\n')

### Загрузка библиотеки FastText

In [ ]:
# чтобы установить fasstext, можно склонировать его с репозитория
%%capture
! git clone https://github.com/facebookresearch/fastText.git
! pip3 install fastText/.

In [ ]:
import fasttext

In [ ]:
%%time
ft_model = fasttext.train_supervised('train.txt', loss='ova')

CPU times: user 34.8 s, sys: 649 ms, total: 35.4 s
Wall time: 35.8 s


In [ ]:
import numpy as np
from google.colab import files

# вычисляет и скачивает файл для submission на kaggle
def submit(model, X_test):
    predictions = []
    for test_string in X_test:
        prediction = ft_model.predict(test_string, k=1)
        predictions.append(int(prediction[-1]))

    df = pd.DataFrame(data = {'id':np.arange(len(predictions)), 'topic_label':predictions})

    df.to_csv('submission.csv', index=False)
    files.download('submission.csv')

In [ ]:
submit(ft_model, X_test)

<ipython-input-12-867836b33cee>:9: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  predictions.append(int(prediction[-1]))


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>